Här ska jag försöka sortera ut alla djurbilder som saknar MD-annoteringar + särskilja bilderna med hundar på på något sätt

In [50]:
import shutil

import os
import sys
import json
import pandas as pd

from pandas import DataFrame

In [109]:
TH = 0.8
OUTPUT_FOLDER = "output/"
IMG_FOLDER = "images/"
json_path = "some_output_file.json"
json_data = json.load(open(json_path))

In [87]:
human_cameras = list()
human_timestamps = list()
human_number = list()
dog_number = list()
human_filenames = list()

animal_cameras = list()
animal_number = list()
animal_timestamps = list()
animal_filenames = list()

In [68]:
img = json_data["images"][0]
filename = (img["file"].split("/")[-1]).split("\\")[-1]
cam = filename[0:2]
det = img["detections"][0]

In [74]:
def camera_name(cam):
    if cam == 'BR':
        return 'Bridge'
    elif cam == 'PA':
        return 'Path'
    elif cam == 'TL':
        return 'Treeline'
    elif cam == 'TR':
        return 'Track'
    elif cam == 'TS':
        return 'TSS'
    elif cam == 'ZZ':
        return 'Zigzag'
    elif cam == 'CP':
        return 'carpark'
    elif cam == 'ST':
        return 'stream'
    elif cam == 'SN':
        return 'SNH'
    else:
        print('Unknown camera: '+ cam )
        return cam

In [88]:
for img in json_data["images"]:
    filename = (img["file"].split("/")[-1]).split("\\")[-1]
    cam = filename[0:2]
    Y = filename[-19:-15]
    M = filename[-15:-13]
    D = filename[-13:-11]
    h = filename[-10:-8]
    m = filename[-8:-6]
    s = filename[-6:-4]
    ts = pd.Timestamp(Y + '-' + M + '-'+ D + ' ' + h + ':' + m + ':' + s)
    
    animals = 0
    humans = 0
    dogs = 0
    
    for det in img["detections"]:
        if det["conf"] >= TH:
            if det["category"] == '1':
                animals +=1
            elif det["category"] == '2':
                humans +=1
        else:
            break
    
    if animals > 0 and humans > 0:
        dogs = animals
        animals = 0
        
    elif animals > 0:
        animal_cameras.append(camera_name(cam)) 
        animal_number.append(animals)
        animal_timestamps.append(ts)
        animal_filenames.append(filename)
        
    if humans > 0 or dogs >0 :
        human_cameras.append( camera_name(cam) )
        human_timestamps.append(ts)
        human_number.append(humans)
        dog_number.append(dogs)
        human_filenames.append(filename)

In [101]:
i=0
while i in range(len(animal_filenames)):
    dog = False
    for j in range(len(human_filenames)):
        if (animal_cameras[i] == human_cameras[j] and 
            abs((animal_timestamps[i]-human_timestamps[j]).total_seconds()) < 300):
            
            dog = True
            human_cameras.append(animal_cameras.pop(i))
            human_timestamps.append(animal_timestamps.pop(i))
            human_number.append(0)
            dog_number.append(animal_number.pop(i))
            human_filenames.append(animal_filenames.pop(i))
            break
    
    if not dog:
        i+=1

In [104]:
animal_df = DataFrame({'Timestamp': animal_timestamps, 'Camera': animal_cameras, 
                      'Number': animal_number})
human_df = DataFrame({'Timestamp': human_timestamps, 'Camera': human_cameras, 
                      'Number': human_number, 'Dogs': dog_number})

animal_df.to_excel(OUTPUT_FOLDER+'wildlife.xlsx', sheet_name='sheet1', index=False)
human_df.to_excel(OUTPUT_FOLDER+'people.xlsx', sheet_name='sheet1', index=False)

In [114]:
if not os.path.exists(OUTPUT_FOLDER+'Empty'):
    os.makedirs(OUTPUT_FOLDER+'Empty')

if not os.path.exists(OUTPUT_FOLDER+'Animal'):
    os.makedirs(OUTPUT_FOLDER+'Animal')
    
if not os.path.exists(OUTPUT_FOLDER+'Human'):
    os.makedirs(OUTPUT_FOLDER+'Human')
# Could add dog-folder

In [115]:
for filename in animal_filenames:
    shutil.move(IMG_FOLDER+filename, OUTPUT_FOLDER+'Animal/'+filename)
    
for filename in human_filenames:
    shutil.move(IMG_FOLDER+filename, OUTPUT_FOLDER+'Human/'+filename)

for filename in os.listdir(IMG_FOLDER):
    shutil.move(IMG_FOLDER+filename, OUTPUT_FOLDER+'Empty/'+filename)

In [24]:
x = 1 # 0 for animals, 1 for humans

folder = folders[x]
pics = os.listdir(folder)
detected = False
dog = False
n_dogs=0
missed = 0

if not os.path.exists("missed"):
    os.mkdir("missed")
    
if not os.path.exists("missed_annotations"):
    os.mkdir("missed_annotations")


for i in range( len(pics) ):
    #path = "annotations/"+ folder + pics[i][:-4]+".txt"
    path = "annotations/2013_yolo/" + pics[i][:-4]+".txt"
    if os.path.exists(path):
        f = open(path, "r")
        lines = f.readlines()
        f.close()
        detected = False
        dog =False
        
        for line in lines:
            if int(line[0]) == x:  #checks if there was an animal/human detected (we don't want vehicle) 
                detected = True
                break
            elif int(line[0]) == 0:
                dog = True
        
        if detected == False and dog : 
            n_dogs +=1
            try:
                a = 1
                shutil.move(folder+pics[i],"dogs_2013/"+ pics[i])
                shutil.move(path,"dogs_2013_annotations/"+ pics[i][:-4]+".txt")
            except:
                print("Exception error")
                print("Couldn't move " + pics[i])
        elif detected == False:
            shutil.move(folder+pics[i] ,"missed/"+ pics[i])
            shutil.move(path,"missed_annotations/"+ pics[i][:-4]+".txt")
            missed+=1
        continue
        
    elif pics[i][-4:] ==".ini" :
        continue
        
    else:
        try:
            shutil.move(folder+pics[i] ,"missed/"+ pics[i])
        except:
            print("Exception error")
            print("Couldn't move " + pics[i])
        missed+=1 #här har den missat att detecta en människa

print("Number of dogs " + str(n_dogs))
print("Number of missed "+ str(missed))

Number of dogs 100


In [26]:
# Remove annotations were there isn't anything to detect
folder = "2013_sorted/Empty/"
pics = os.listdir(folder)

missed = 0

for i in range( len(pics) ):
    path = "annotations/2013_yolo/" + pics[i][:-4]+".txt"
    if os.path.exists(path):
        f = open(path, "r")
        lines = f.readlines()
        f.close()
        
        for line in lines:
            if int(line[0]) == 0 or int(line[0]) == 1: 
                shutil.move(folder+pics[i],"not_empty/"+ pics[i])
                shutil.move(path ,"not_empty_annotations/"+ pics[i][:-4]+".txt")
                missed+=1
                break
        
    elif pics[i][-4:] ==".ini" :
        continue

print("Potentially false positives: "+ str(missed))

Potentially false positives: 104


In [113]:
for file in os.listdir(IMG_FOLDER):
    print(file)

ZZ 20130101 143354.JPG
ZZ 20130101 144659.JPG
ZZ 20131012 133533.JPG
